In [29]:
import numpy as np
import gym

import torch
import json
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim, distributions
import cv2

import random                 # Handling random number generation
import time                   # Handling time calculation
from skimage import transform # Help us to preprocess the frames

from collections import deque # Ordered collection with ends
import matplotlib.pyplot as plt  # Display graphs

import warnings                  # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore')
import os


### MODEL HYPERPARAMETERS
learning_rate =  1e-4   # Alpha (aka learning rate)
total_episodes = 100000

### TRAINING HYPERPARAMETERS
total_episodes = 100000         # Total episodes for training

clip_norm = 0.0001
batch_size = 64
total_timesteps = 1000000

model_folder = f"./models/reinforce/CN{clip_norm}_LR{learning_rate}_B{batch_size}"
os.makedirs(model_folder)
log_path = f"./logs/reinforce/CN{clip_norm}_LR{learning_rate}_B{batch_size}.txt"

env = gym.make('CartPole-v1')

class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4, 32)
        self.layer2 = nn.Linear(32, 2)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, state):
        x = F.relu(self.layer1(state))
        x = self.layer2(x)
        return x
    
model = Net()
opt = optim.Adam(params=model.parameters(), lr=learning_rate)




In [30]:


class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4, 32)
        self.layer2 = nn.Linear(32, 2)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, state):
        x = F.relu(self.layer1(state))
        x = self.layer2(x)
        return x
    
model = Net()
opt = optim.Adam(params=model.parameters(), lr=learning_rate)

out_f = open(log_path, 'w')

total_steps = 0

# Initialise batch parameters
batch_loss = 0.0
batch_reward = 0.0

for episode in range(total_episodes):

    action_log_probs = []

    episode_entropy = 0.0
    episode_steps = 0
    
    episode_reward = 0

    episode_argmax_action_taken = 0

    state, _ = env.reset()

    for time_steps in range(10000):

        # choose state from model
        tensor_state = torch.from_numpy(state).float().unsqueeze(0)
        action_logits = model(tensor_state)
        action_probs = F.softmax(action_logits, dim=1)

        # print(episode_entropy)
        # asdf
        m = distributions.Categorical(action_probs)
        action = m.sample()
        log_prob = m.log_prob(action)
        

        action_log_probs.append(log_prob)
        action = action.item()
        
        # _, argmax_action = action_probs.max(dim=-1)

        # argmax_action = argmax_action.item()
        # if argmax_action == action:
        #     episode_argmax_action_taken += 1

        # Makes an action (here random one) and returns a reward.
        next_state, reward, done, _, _ = env.step(action)
        episode_steps += 1
        total_steps += 1
        episode_reward += reward

        state = next_state

        if done:
            per_timestep_losses = [- log_prob * episode_reward for log_prob in action_log_probs]

            per_timestep_losses_t = torch.cat(per_timestep_losses)
            loss = per_timestep_losses_t.sum()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)

            # batch gradient 

            batch_loss += loss.item()
            batch_reward += episode_reward
            # batch_argmax_action_prop += episode_argmax_action_prop
            

            if (episode + 1) % batch_size == 0:
                # batch_avg_reward = batch_reward / batch_size
                # batch_avg_loss = batch_loss / batch_size
                # batch_avg_argmax_action_prop = batch_argmax_action_prop / batch_size
                # torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
                opt.step()
                opt.zero_grad()

                batch_loss = 0.0
                batch_reward = 0.0
                batch_argmax_action_prop = 0.0


            print(f"Episode {episode}     |      Reward: {episode_reward}     |      Total timesteps: {total_steps}")

            out_f.write(json.dumps({
                'episode': episode,
                'reward': episode_reward,
                }) + '\n')
            
            out_f.flush()

            if episode % 100 == 0:
                torch.save(model, f'{model_folder}/E_{episode}.pt')
                print(f'====== Model saved ======')
            break

    if total_steps > total_timesteps:
        break

env.close()



Episode 0     |      Reward: 30.0     |      Total timesteps: 30
====== Model saved ======
Episode 1     |      Reward: 31.0     |      Total timesteps: 61
Episode 2     |      Reward: 17.0     |      Total timesteps: 78
Episode 3     |      Reward: 40.0     |      Total timesteps: 118
Episode 4     |      Reward: 15.0     |      Total timesteps: 133
Episode 5     |      Reward: 18.0     |      Total timesteps: 151
Episode 6     |      Reward: 13.0     |      Total timesteps: 164
Episode 7     |      Reward: 14.0     |      Total timesteps: 178
Episode 8     |      Reward: 20.0     |      Total timesteps: 198
Episode 9     |      Reward: 19.0     |      Total timesteps: 217
Episode 10     |      Reward: 50.0     |      Total timesteps: 267
Episode 11     |      Reward: 22.0     |      Total timesteps: 289
Episode 12     |      Reward: 20.0     |      Total timesteps: 309
Episode 13     |      Reward: 25.0     |      Total timesteps: 334
Episode 14     |      Reward: 50.0     |      Tot